<a href="https://colab.research.google.com/github/Seb207/Market-Context-Project/blob/main/Irrationality%20Index/DL_Index_Outline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning for Index Outline

Outline for deep learning model to analze market factors and output irrationality index.

Model: Transformer

Data_Format: history data in second

- Positional Embedding Class
- Transformer Encoder Block Class
- Prediction Head Class
- Irrationality Index Class

# Hypothesis

**Factors**
- Volatility: VIX (moving avg. - 20 days)
- Volume: Trading Volume (periodic average - 20 days)
- Fixed Income (macro): US bond rate momentum
- Periodicity: past rate of return (20, 60, 120 days)
- Small vs. Big performace: (QQQ -> factor: IWM, IWM -> factor: QQQ) rate of return
-

**Tested Data**
- SPY (S&P 500)
- QQQ (Nasdaq)
- DIA (Dow Jones)
- IWM (Russell 2000)
- Individual stocks

1.

# Data Preprocessing

Problem 1:
- As factors have each different absolute values, output could be affected by its absolute value if directly used as input.

Solution:
- Used data normalization (z-score) to

# Import Libraries

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import sklearn as skl
import matplotlib.pyplot as plt

# Hyperparameters

**Model Architecture**
- input_dim: number of factors
- seq_len: length of data processed in the model at once
- d_model: dimension of the model
- num_heads: number of heads for multi-head attention (should be divisor of d_model)
- d_ff: inner dimension of feed forward (4x of d_model in usual)
- num_layers: number of transformer encoder block (depth of model)
- output_dim: output dimension

**Training and Normalization**
- dropout_rate
- learning_rate
- batch_size

# Transformer Encoder Block Class





In [ ]:
class TransformerEncoderBlock(layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=d_model // num_heads, dropout=dropout_rate
        )
        self.feed_forward = keras.Sequential([
            layers.Dense(d_ff, activation="relu"),
            layers.Dense(d_model),
        ])
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout_layer = layers.Dropout(dropout_rate)

    def call(self, inputs, training=False, mask=None):
        attn_output = self.attention(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=mask,
            training=training,
        )
        x1 = self.norm1(inputs + attn_output)

        ffn_output = self.feed_forward(x1)
        ffn_output = self.dropout_layer(ffn_output, training=training)

        return self.norm2(x1 + ffn_output)

# Positional Embedding Class

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, d_model, **kwargs):
        super().__init__(**kwargs)
        self.sequence_length = sequence_length
        self.d_model = d_model

        position_indices = tf.range(start=0, limit=sequence_length, delta=1)
        angles = tf.range(start=0, limit=d_model, delta=1, dtype="float32")
        angles = 1 / tf.pow(10000.0, (angles - angles % 2) / d_model)

        position_indices = tf.cast(tf.expand_dims(position_indices, -1), "float32")
        angles = tf.cast(tf.expand_dims(angles, 0), "float32")

        positional_encoding = position_indices * angles
        positional_encoding = tf.where(tf.range(d_model) % 2 == 0,
                                       tf.sin(positional_encoding),
                                       tf.cos(positional_encoding))

        self.positional_encoding = tf.cast(positional_encoding, "float32")

    def call(self, inputs):
        return inputs + self.positional_encoding

# Prediction Head Class

In [4]:
class PredictionHead(keras.Model):
    def __init__(self, d_model, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.dense1 = layers.Dense(d_model // 2, activation="relu")
        self.dense2 = layers.Dense(output_dim)

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

# Irrationality Index Class

In [ ]:
class IrrationalityIndex(keras.Model):
    def __init__(self, input_dim, d_model, seq_len, num_heads, d_ff, num_layers, output_dim, **kwargs):
        super().__init__(**kwargs)

        self.factor_weights = layers.Dense(input_dim, activation='sigmoid')
        self.normalize = layers.Normalization(axis=-1)
        self.input_projection = layers.Dense(d_model)
        self.pos_embedding = PositionalEmbedding(seq_len, d_model)
        self.dropout = layers.Dropout(0.1)

        self.encoder_blocks = [
            TransformerEncoderBlock(d_model, num_heads, d_ff, 0.1)
            for _ in range(num_layers)
        ]

        self.prediction_head = PredictionHead(d_model, output_dim)

    def call(self, inputs, training=False):
        inputs_scaled = self.normalize(inputs)
        weights = self.factor_weights(inputs_scaled)
        weighted_inputs = inputs_scaled * weights

        x = self.input_projection(weighted_inputs)
        x = self.pos_embedding(x)
        x = self.dropout(x, training=training)

        attention_mask = tf.cast(tf.math.not_equal(inputs, 0), dtype=tf.float32)
        attention_mask = tf.expand_dims(attention_mask, axis=1)

        for block in self.encoder_blocks:
            x = block(x, training=training, mask=attention_mask)

        context_vector = x
        outputs = self.prediction_head(context_vector)

        return outputs

# Training Data

In [ ]:
# Example of parameters
input_features = 5
seq_length = 30
d_model = 128
num_heads = 4
d_ff = 512
num_layers = 3
output_classes = 2

In [ ]:
model = IrrationalityIndex(input_features, d_model, seq_length, num_heads, d_ff, num_layers, output_classes)

In [ ]:
train_data = np.random.randn(100, seq_length, input_features).astype(np.float32) # randomly generated for example

In [ ]:
model.normalize.adapt(train_data) # normalize to avoid error due to difference in absolute values

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])